In [10]:
import os, re, glob
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

import numpy as np
import pandas as pd
import tensorflowjs as tfjs

from keras import layers, Model, callbacks
from keras.optimizers import Adam
from keras.applications.mobilenet import MobileNet
from keras.preprocessing.image import ImageDataGenerator
from ohia.metrics import top_1_accuracy, top_3_accuracy, top_5_accuracy
from ohia.encoders import FastLabelEncoder

In [3]:
# Set parameters
SEED = 0
BATCH_SIZE = 32
MODEL_NAME = 'mobilenetv1'
FILE_PATH = '/home/matt/repos/ohia.ai/data'
IMAGE_DIR = 'preprocessed_images'

In [17]:
# get list of images and labels
file_list = glob.glob(f'{FILE_PATH}/{IMAGE_DIR}/**/*.jpg', recursive=True)
label_list = [re.split('/', f)[-2] for f in file_list]
label_ids = FastLabelEncoder().fit_transform(label_list)
n_classes = len(np.unique(label_ids))

In [18]:
# create generator
data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    brightness_range=(0.75, 1.25),
    horizontal_flip=True,
    validation_split=0.1
)

training_generator = data_generator.flow_from_directory(
    f'{FILE_PATH}/{IMAGE_DIR}',
    target_size=(224, 224),
    shuffle=True,
    seed=SEED,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    subset='training'
)

validation_generator = data_generator.flow_from_directory(
    f'{FILE_PATH}/{IMAGE_DIR}',
    target_size=(224, 224),
    shuffle=True,
    seed=SEED,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 10593 images belonging to 23 classes.
Found 1163 images belonging to 23 classes.


In [19]:
# load pretrained ImageNet model
base_model = MobileNet(
    input_shape=(224,224,3),
    weights='imagenet',
    include_top=False
)

# set freeze all layers
for layer in base_model.layers:
    layer.trainable = False
    
# map ImageNet features to 39 plants
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
outputs = layers.Dense(n_classes, activation='softmax')(x)
    
# compile the model
model = Model(inputs=base_model.input, outputs=outputs)
model.compile(
    optimizer=Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=[top_1_accuracy, top_3_accuracy, top_5_accuracy]
)

In [20]:
# define callbacks
callbacks_list = [
    callbacks.EarlyStopping(
        monitor='val_top_3_accuracy',
        patience=10,
        verbose=1,
        mode='max',
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_top_3_accuracy',
        factor=0.5,
        patience=3,
        verbose=1,
        mode='max',
    ),
    callbacks.ModelCheckpoint(
        monitor='val_top_3_accuracy',
        filepath=f'{FILE_PATH}/weights/{MODEL_NAME}.h5',
        save_best_only=True,
        save_weights_only=True,
        mode='max',
    ),
]

# train model
model.fit_generator(
    generator=training_generator,
    validation_data=validation_generator,
    steps_per_epoch=round(training_generator.n/BATCH_SIZE),
    validation_steps=round(validation_generator.n/BATCH_SIZE),
#     use_multiprocessing=True, workers=20
    epochs=100,    
)

Epoch 1/100
331/331 [==============================] - 137s 413ms/step - loss: 1.8012 - top_1_accuracy: 0.4906 - top_3_accuracy: 0.7011 - top_5_accuracy: 0.7910 - val_loss: 2.0005 - val_top_1_accuracy: 0.4097 - val_top_3_accuracy: 0.6615 - val_top_5_accuracy: 0.7821
Epoch 2/100
331/331 [==============================] - 135s 407ms/step - loss: 1.2097 - top_1_accuracy: 0.6381 - top_3_accuracy: 0.8393 - top_5_accuracy: 0.9064 - val_loss: 1.9442 - val_top_1_accuracy: 0.4580 - val_top_3_accuracy: 0.6914 - val_top_5_accuracy: 0.7984
Epoch 3/100
331/331 [==============================] - 131s 394ms/step - loss: 1.0463 - top_1_accuracy: 0.6869 - top_3_accuracy: 0.8695 - top_5_accuracy: 0.9305 - val_loss: 1.9229 - val_top_1_accuracy: 0.4642 - val_top_3_accuracy: 0.7233 - val_top_5_accuracy: 0.8187
Epoch 4/100
331/331 [==============================] - 131s 397ms/step - loss: 0.9381 - top_1_accuracy: 0.7199 - top_3_accuracy: 0.8871 - top_5_accuracy: 0.9442 - val_loss: 1.8956 - val_top_1_accurac

KeyboardInterrupt: 

In [26]:
# save model
tfjs.converters.save_keras_model(model, f'models/{MODEL_NAME}/tfjs_model')